tgb - 11/13/2019 - Testing if rescaling the inputs changes the ability of the network to generalize.   
Architecture = From [], predict []   
Test 1 = Rephrase all water concentrations as RH  
Test 2 = Rephrase all temperature as deviations from moist adiabat  
  
tgb - 11/15/2019 - This notebook is mostly used for pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog
[Errno 2] No such file or directory: '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM'
/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# 1) Preprocessing

In [2]:
%cd /export/nfs0home/tbeucler/CBRAIN-CAM

/nfspool-0/home/tbeucler/CBRAIN-CAM


In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/09/2021 12:57:46 PM Preprocess training dataset including output quantiles
04/09/2021 12:57:46 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_09_PERC_TRAIN_M4K.nc
04/09/2021 12:57:46 PM Reading input files
04/09/2021 12:57:46 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0001-0[1-4]-*-*.nc
04/09/2021 12:57:53 PM Reading PKL file containing univariate distribution of output variables
04/09/2021 12:57:53 PM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_03_18_O3_TRAIN_M4K_shuffle.nc_PERC.pkl
04/09/2021 12:57:53 PM Crop levels
04/09/2021 12:57:53 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQPERC
Interpolating level  0 out of  30
Interpolating level  1 out of  30
Interpolating level  2 out of  30
Interpolating level  3 out of  30
Interpolating le

OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146080 thread 29 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146073 thread 30 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146075 thread 31 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146074 thread 32 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146061 thread 33 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146069 thread 34 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146042 thread 35 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146071 thread 36 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146081 thread 37 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146083 thread 38 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146048 thread 39 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 145631 tid 146070 thread 40 bou

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test.yml

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_03_31_O3_T_NSto220_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/31/2021 12:45:59 PM Preprocess training dataset
03/31/2021 12:45:59 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_31_O3_T_NSto220_small.nc
03/31/2021 12:45:59 PM Reading input files
03/31/2021 12:45:59 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/31/2021 12:46:23 PM Crop levels
03/31/2021 12:46:23 PM Create stacked dataarray
var is  QBP
var is  T_NSto220
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: [47]
03/31/2021 12:46:25 PM Stack and reshape dataarray
03/31/2021 12:46:26 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_31_O3_T_NSto220_small.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #15

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/18/2021 04:24:21 PM Preprocess training dataset
03/18/2021 04:24:21 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TRAIN_P4K.nc
03/18/2021 04:24:21 PM Reading input files
03/18/2021 04:24:21 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_4k/sp8fbp_4k.cam2.h2.0001-0[1-4]-*-*.nc
03/18/2021 04:29:31 PM Crop levels
03/18/2021 04:29:31 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
03/18/2021 04:29:59 PM Stack and reshape dataarray
03/18/2021 04:30:28 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TRAIN_P4K.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OM

In [4]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/18/2021 06:58:23 PM Preprocess training dataset
03/18/2021 06:58:23 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TEST_P4K.nc
03/18/2021 06:58:23 PM Reading input files
03/18/2021 06:58:23 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_4k/sp8fbp_4k.cam2.h2.0000-0[6-9]-*-*.nc
03/18/2021 07:03:19 PM Crop levels
03/18/2021 07:03:19 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
03/18/2021 07:03:48 PM Stack and reshape dataarray
03/18/2021 07:04:21 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TEST_P4K.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP:

In [5]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/18/2021 08:08:28 PM Preprocess training dataset
03/18/2021 08:08:28 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TRAIN_M4K.nc
03/18/2021 08:08:28 PM Reading input files
03/18/2021 08:08:28 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0001-0[1-4]-*-*.nc
03/18/2021 08:13:16 PM Crop levels
03/18/2021 08:13:16 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
03/18/2021 08:13:44 PM Stack and reshape dataarray
03/18/2021 08:14:12 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TRAIN_M4K.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respe

In [6]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/18/2021 10:15:57 PM Preprocess training dataset
03/18/2021 10:15:57 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TEST_M4K.nc
03/18/2021 10:15:57 PM Reading input files
03/18/2021 10:15:57 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0000-0[6-9]-*-*.nc
03/18/2021 10:21:16 PM Crop levels
03/18/2021 10:21:16 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
03/18/2021 10:21:46 PM Stack and reshape dataarray
03/18/2021 10:22:17 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TEST_M4K.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respect

In [22]:
!python preprocessing-11132019.py -c pp_config/2021_02_01_O3_BCONS_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/17/2021 11:14:59 AM Preprocess training dataset
03/17/2021 11:14:59 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_17_O3_BCONS_small.nc
03/17/2021 11:14:59 AM Reading input files
03/17/2021 11:14:59 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/17/2021 11:14:59 AM Crop levels
03/17/2021 11:14:59 AM Create stacked dataarray
var is  QBP
var is  BCONS
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS pro

In [5]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/15/2021 09:55:54 PM Preprocess training dataset
03/15/2021 09:55:54 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_LHF_nsDELQ_small.nc
03/15/2021 09:55:54 PM Reading input files
03/15/2021 09:55:54 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/15/2021 09:55:54 PM Crop levels
03/15/2021 09:55:54 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHF_nsDELQ
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_A

In [5]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/17/2021 06:48:04 PM Preprocess training dataset
03/17/2021 06:48:04 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_17_O3_LHF_nsQ_small.nc
03/17/2021 06:48:04 PM Reading input files
03/17/2021 06:48:04 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/17/2021 06:48:04 PM Crop levels
03/17/2021 06:48:04 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHF_nsQ
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: [47]
03/17/2021 06:48:05 PM Stack and reshape dataarray
03/17/2021 06:48:06 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_17_O3_LHF_nsQ_small.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_A

In [8]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/01/2021 10:37:41 PM Preprocess training dataset
03/01/2021 10:37:41 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_LHF_nsDELQ_small.nc
03/01/2021 10:37:41 PM Reading input files
03/01/2021 10:37:41 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/01/2021 10:37:42 PM Crop levels
03/01/2021 10:37:42 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHF_nsDELQ
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_A

In [9]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/01/2021 10:38:29 PM Preprocess training dataset
03/01/2021 10:38:29 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_LHF_nsQ_small.nc
03/01/2021 10:38:29 PM Reading input files
03/01/2021 10:38:29 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/01/2021 10:38:30 PM Crop levels
03/01/2021 10:38:30 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHF_nsQ
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_AFFINIT

In [30]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_02_01_O3_QSATdeficit_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
02/25/2021 08:38:08 PM Preprocess training dataset
02/25/2021 08:38:08 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_QSATdeficit_small.nc
02/25/2021 08:38:08 PM Reading input files
02/25/2021 08:38:08 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
02/25/2021 08:38:10 PM Crop levels
02/25/2021 08:38:10 PM Create stacked dataarray
var is  QSATdeficit
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS pro

In [13]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_input_small_data_test

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/24/2021 03:32:52 PM Preprocess training dataset
01/24/2021 03:32:52 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_small.nc
01/24/2021 03:32:52 PM Reading input files
01/24/2021 03:32:52 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
01/24/2021 03:32:53 PM Crop levels
01/24/2021 03:32:53 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: [47]
01/24/2021 03:32:54 PM Stack and reshape dataarray
01/24/2021 03:32:55 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_small.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial O

In [14]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Train_Valid.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/24/2021 03:46:36 PM Preprocess training dataset
01/24/2021 03:46:36 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TRAIN.nc
01/24/2021 03:46:36 PM Reading input files
01/24/2021 03:46:36 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
01/24/2021 03:50:19 PM Crop levels
01/24/2021 03:50:19 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
01/24/2021 03:50:59 PM Stack and reshape dataarray
01/24/2021 03:51:35 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TRAIN.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc 

In [30]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/24/2021 06:48:24 PM Preprocess training dataset
01/24/2021 06:48:24 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TEST.nc
01/24/2021 06:48:24 PM Reading input files
01/24/2021 06:48:24 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-0[6-9]-*-*.nc
01/24/2021 06:52:06 PM Crop levels
01/24/2021 06:52:06 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
01/24/2021 06:52:55 PM Stack and reshape dataarray
01/24/2021 06:53:31 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TEST.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc se

In [1]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

python: can't open file 'preprocessing-11132019.py': [Errno 2] No such file or directory


In [4]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/118_POG_TfromTSinput_test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [13]:
!ls -la -h /local/Tom.Beucler/SPCAM_PHYS/1*

-rw-r--r-- 1 Tom.Beucler ls-craig  15K Nov 13 22:17 /local/Tom.Beucler/SPCAM_PHYS/100_norm.nc
-rw-r--r-- 1 Tom.Beucler ls-craig 1.8G Nov 13 22:16 /local/Tom.Beucler/SPCAM_PHYS/100_train.nc
-rw-r--r-- 1 Tom.Beucler ls-craig 1.8G Nov 13 22:16 /local/Tom.Beucler/SPCAM_PHYS/100_train_shuffle.nc
-rw-r--r-- 1 Tom.Beucler ls-craig 1.8G Nov 13 22:17 /local/Tom.Beucler/SPCAM_PHYS/100_valid.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 22:32 /local/Tom.Beucler/SPCAM_PHYS/101_train.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 22:33 /local/Tom.Beucler/SPCAM_PHYS/101_train_shuffle.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 22:47 /local/Tom.Beucler/SPCAM_PHYS/101_valid.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 23:57 /local/Tom.Beucler/SPCAM_PHYS/102_train.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 23:58 /local/Tom.Beucler/SPCAM_PHYS/102_train_shuffle.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 14 00:13 /local/Tom.Beucler/SPCAM_PHYS/102_valid.nc
-rw-r--r-- 1 Tom.Beucler ls

# 2) Check the preprocessed data

In [23]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [24]:
path_QSATdeficit = path_data+'2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [25]:
path_TfromNS = path_data+'2021_02_01_NORM_O3_TfromNS_small.nc'

In [26]:
path_Bcons = path_data+'2021_03_17_NORM_O3_BCONS_small.nc'

In [27]:
data_QSATdeficit = xr.open_dataset(path_QSATdeficit)
data_TfromNS = xr.open_dataset(path_TfromNS)
data_Bcons = xr.open_dataset(path_Bcons)

In [9]:
data_QSATdeficit

<xarray.Dataset>
Dimensions:           (var_names: 214, var_names_single: 11)
Coordinates:
  * var_names         (var_names) object 'QSATdeficit' 'QSATdeficit' ... 'QRS'
  * var_names_single  (var_names_single) object 'QSATdeficit' 'TBP' ... 'QRS'
Data variables:
    mean              (var_names) float64 ...
    std               (var_names) float64 ...
    min               (var_names) float64 ...
    max               (var_names) float64 ...
    std_by_var        (var_names_single) float64 ...

In [17]:
data_TfromNS

<xarray.Dataset>
Dimensions:           (var_names: 214, var_names_single: 11)
Coordinates:
  * var_names         (var_names) object 'QBP' 'QBP' 'QBP' ... 'QRS' 'QRS' 'QRS'
  * var_names_single  (var_names_single) object 'QBP' 'TfromNS' ... 'QRL' 'QRS'
Data variables:
    mean              (var_names) float64 ...
    std               (var_names) float64 ...
    min               (var_names) float64 ...
    max               (var_names) float64 ...
    std_by_var        (var_names_single) float64 ...

In [23]:
data_TfromNS['mean'].shape

(214,)

In [10]:
data_QSATdeficit['mean'][:30]

<xarray.DataArray 'mean' (var_names: 30)>
array([3.514103e-02, 5.407069e-03, 3.441649e-04, 2.163555e-03, 6.659779e-04,
       2.984441e-04, 1.824647e-04, 1.344675e-04, 9.427601e-05, 6.970832e-05,
       5.265598e-05, 3.859004e-05, 3.558415e-05, 5.230486e-05, 1.046076e-04,
       2.272342e-04, 4.811124e-04, 9.102161e-04, 1.484617e-03, 2.062735e-03,
       2.350633e-03, 1.879407e-03, 9.880273e-04, 7.757872e-04, 6.943080e-04,
       7.268760e-04, 9.064377e-04, 1.236276e-03, 1.673174e-03, 1.837612e-03])
Coordinates:
  * var_names  (var_names) object 'QSATdeficit' 'QSATdeficit' ... 'QSATdeficit'

In [24]:
data_TfromNS['mean'][30:60]

<xarray.DataArray 'mean' (var_names: 30)>
array([-59.558428, -66.212014, -81.912259, -64.22248 , -67.529307, -72.369765,
       -76.540285, -77.553016, -77.075974, -75.904117, -74.379141, -72.655088,
       -70.094929, -66.142031, -60.618139, -53.835679, -46.212485, -38.179249,
       -30.09733 , -22.926994, -17.293411, -13.378349, -10.817899,  -8.781711,
        -7.18645 ,  -5.597677,  -4.051676,  -2.564866,  -1.248037,   0.      ])
Coordinates:
  * var_names  (var_names) object 'TfromNS' 'TfromNS' ... 'TfromNS' 'TfromNS'

In [31]:
data_Bcons['mean'][30:60]

<xarray.DataArray 'mean' (var_names: 30)>
array([-7.533478e+00, -6.630077e+00, -5.619210e+00, -5.348259e+00,
       -4.655388e+00, -3.962523e+00, -3.336569e+00, -2.922490e+00,
       -2.624306e+00, -2.338432e+00, -2.051976e+00, -1.759623e+00,
       -1.487952e+00, -1.260196e+00, -1.083615e+00, -9.475950e-01,
       -8.369946e-01, -7.347770e-01, -6.247500e-01, -5.009889e-01,
       -3.754159e-01, -2.512172e-01, -1.457106e-01, -9.167787e-02,
       -6.140617e-02, -3.838775e-02, -2.120094e-02, -9.338597e-03,
        9.624067e-04,  5.734316e-03])
Coordinates:
  * var_names  (var_names) object 'BCONS' 'BCONS' 'BCONS' ... 'BCONS' 'BCONS'

In [32]:
data_Bcons['std'][30:60]

<xarray.DataArray 'std' (var_names: 30)>
array([0.561294, 0.383327, 0.394672, 0.437875, 0.41099 , 0.503552, 0.666199,
       0.723019, 0.669053, 0.592892, 0.502611, 0.390778, 0.28052 , 0.202901,
       0.175193, 0.194535, 0.216873, 0.218058, 0.199232, 0.171828, 0.142094,
       0.107694, 0.074768, 0.060203, 0.050628, 0.041717, 0.033625, 0.02606 ,
       0.018296, 0.014811])
Coordinates:
  * var_names  (var_names) object 'BCONS' 'BCONS' 'BCONS' ... 'BCONS' 'BCONS'

# 3) Preprocessing for PRL paper

In [2]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-RH-test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [3]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages